In [1]:


from moleculekit.molecule import Molecule
import numpy as np

topology_psf_path = '../data/CG/structure.psf'
mol = Molecule(topology_psf_path)

arr = np.load("../data/CG/pos.npy")
mol.coords = np.moveaxis(arr, 0, -1)
mol.box = np.load("../data/CG/box.npy")

priors = {}
priors['atomtypes'] = list(set(mol.atomtype))

Temp = 300
kB = 0.0019872041  # Boltzmann constant in kcal/mol/K


In [11]:
from scipy.optimize import curve_fit

def multi_harmonic_dihedral(x, *params):
    # Calculate the multi-harmonic dihedral energy
    n_harmonic = len(params)
    E = 0
    for i in range(n_harmonic):
        E += params[i] * np.cos(np.radians(i * x))**i
    return E

def renorm(counts, bins):
    """ Renormalize counts by the volume of spherical shells and return bin centers with normalized counts. """
    R = 0.5 * (bins[1:] + bins[:-1])  # Bin centers
    vols = 4 * np.pi / 3 * (bins[1:]**3 - bins[:-1]**3)
    ncounts = counts / vols
    return np.vstack([R, ncounts])

def minimum_image_distance(r_ij, box_size):
    box_size = box_size.T
    """ Compute the minimum image distance between two points in a periodic box. """
    return r_ij - box_size * np.round(r_ij / box_size)

def get_param_repulsion(mol, cutoff=16, num_bins=200):
    box_size = mol.box
    coords = mol.coords
    
    # Group atoms by atom type
    atom_types = {}
    for at in set(mol.atomtype):
        atom_types[at] = np.where(mol.atomtype == at)[0]
    
    # Initialize a dictionary to hold distance data for each atom type
    prior_repulsion = {}

    # Loop over each atom type
    for at in atom_types.keys():
        dists = []
        
        # Loop over all atoms of this type
        for idx in atom_types[at]:
            
            # Get the segid of the current atom
            segid = mol.segid[idx]
            
            # Loop over all other atoms
            for idx2 in atom_types[at]:
                # Skip the same atom or atoms in the same segid
                if idx != idx2 and mol.segid[idx2] != segid:
                    
                    # Calculate the minimum image distance
                    dist_vec = minimum_image_distance(coords[idx, :, :] - coords[idx2, :, :], box_size)
                    
                    # Calculate the norm (Euclidean distance) and add to the list
                    dist = np.linalg.norm(dist_vec, axis=0)
                    dists.append(dist)
        
        # Concatenate all distances for this atom type
        dist = np.concatenate(dists, axis=0)
    
        dist = dist[dist < cutoff]

        yb, bins = np.histogram(dist, bins=num_bins, range=(0,cutoff))  ### Adjust the range if needed
        RR, ncounts = renorm(yb, bins)

        RR_nz = RR[ncounts > 0]
        ncounts_nz = ncounts[ncounts > 0]
        dG_nz = -kB * Temp * np.log(ncounts_nz)
        
        def Repulsion(r,eps,V0):
            V = 4*eps*(r**-6)+V0
            return V
        
        popt, _ = curve_fit(Repulsion, RR_nz, dG_nz, p0=[0, 10], 
                            maxfev=1000000)
        
        bname = at
        prior_repulsion[bname] = {'epsilon': popt[0].tolist(),
                           'V0': popt[1].tolist()}

        
    return prior_repulsion

def get_param_bond(mol,n_gaussians = 1,num_bins = 180):
    box_size = mol.box
    bonds_types = {}
    for bond in mol.bonds:
        btype = tuple(mol.atomtype[bond])
        if btype in bonds_types:
            bonds_types[btype].append(bond)
        elif tuple([btype[1], btype[0]]) in bonds_types:
            bonds_types[tuple([btype[1], btype[0]])].append(bond)
        else:
            bonds_types[btype] = [bond]

    prior_bond = {}

    for bond in bonds_types.keys():
        dists = []
        for idx0, idx1 in bonds_types[bond]:
            dists.append(np.linalg.norm(minimum_image_distance(mol.coords[idx0, :, :] - mol.coords[idx1, :, :], box_size), axis=0))

        dist = np.concatenate(dists, axis=0)
        bond_range = [np.min(dist), np.max(dist)]
        counts, bins= np.histogram(dist, bins=num_bins,  range=bond_range)
        # Filter out zero counts
        mask = counts > 0
        p_l = counts / counts.sum()
        bins_center = 0.5 * (bins[1:] + bins[:-1])
        p_l = p_l[mask]
        bins_center = bins_center[mask]
        dG_nz = -kB*Temp*np.log(p_l/(bins_center**2))
        
        def gaussian_bond(r, *params):
            n = len(params) // 3
            Ai = params[:n]
            ri = params[n:2*n]
            wi = params[2*n:3*n]
            summation = 0
            for i in range(n):
                summation += Ai[i] / (wi[i] * np.sqrt(np.pi / 2)) * np.exp(-2 * ((r - ri[i]) ** 2) / (wi[i] ** 2))
            return -kB * Temp * np.log(summation)
            
        initial_guess = [1] * (3*n_gaussians)
        
        popt, _ = curve_fit(gaussian_bond, bins_center, dG_nz,p0=initial_guess,
                            maxfev =1000000
                                    )

        bname = f"({bond[0]}, {bond[1]})"
        prior_bond[bname] = {'A_i': popt[:n_gaussians].tolist(),
                            'r_i': popt[n_gaussians:2*n_gaussians].tolist(),
                            'w_i': popt[2*n_gaussians:3*n_gaussians].tolist()}
        
    return prior_bond

def get_param_angle(mol,n_gaussians = 3,num_bins=180):
    box_size = mol.box
    angles_types = {}
    # Collect angles based on triplets of atoms
    for angle in mol.angles:
        atype = tuple(mol.atomtype[list(angle)])
        if atype in angles_types:
            angles_types[atype].append(angle)
        elif tuple(reversed(atype)) in angles_types:
            angles_types[tuple(reversed(atype))].append(angle)
        else:
            angles_types[atype] = [angle]

    prior_angle = {}

    for angle in angles_types.keys():
        angle_vals = []
        for idx0, idx1, idx2 in angles_types[angle]:
            # Compute angle using vector operations
            vec1 = minimum_image_distance(mol.coords[idx0, :, :] - mol.coords[idx1, :, :], box_size)
            vec2 = minimum_image_distance(mol.coords[idx2, :, :] - mol.coords[idx1, :, :], box_size)
            cosine_angle = (vec1 * vec2).sum(axis=0) / (np.linalg.norm(vec1, axis=0) * np.linalg.norm(vec2, axis=0))
            angle_vals.append(np.arccos(np.clip(cosine_angle, -1.0, 1.0)) * (180 / np.pi))  # in degrees

        # Histogram of angle distributions
        angle_range = [np.min(angle_vals), np.max(angle_vals)]
        counts, bins = np.histogram(angle_vals, bins=num_bins, range=angle_range)
        # Filter out zero counts
        mask = counts > 0

        p_theta = counts / counts.sum()
        bins_center = 0.5 * (bins[1:] + bins[:-1])
        p_theta = p_theta[mask]
        bins_center = bins_center[mask]
        dG_nz = -kB * Temp * np.log(p_theta / np.sin(np.radians(bins_center)))
        
        def gaussian_angle(theta, *params):
            n = len(params) // 3
            Ai = params[:n]
            thetai = params[n:2*n]
            wi = params[2*n:3*n]
            summation = 0
            for i in range(n):
                summation += Ai[i] / (wi[i] * np.sqrt(np.pi / 2)) * np.exp(-2 * ((np.radians(theta) - np.radians(thetai[i])) ** 2) / (wi[i] ** 2))
            return -kB * Temp * np.log(summation)

        initial_guess = [1] * (3*n_gaussians)
        popt, _ = curve_fit(gaussian_angle, bins_center, dG_nz, p0=initial_guess,maxfev =100000,
                            )
        
        aname = f"({angle[0]}, {angle[1]}, {angle[2]})"
        prior_angle[aname] = {'A_i': popt[:n_gaussians].tolist(),
                            'theta_i': popt[n_gaussians:2*n_gaussians].tolist(),
                            'w_i': popt[2*n_gaussians:3*n_gaussians].tolist()}
    return prior_angle

def get_param_dihedral(mol,n_harmonic = 8, num_bins=360):
    box_size = mol.box
    dihedral_types = {}
    for dihedral in mol.dihedrals:
        dtype = tuple(mol.atomtype[dihedral])
        if dtype in dihedral_types:
            dihedral_types[dtype].append(dihedral)
        elif tuple(reversed(dtype)) in dihedral_types:
            dihedral_types[tuple(reversed(dtype))].append(dihedral)
        else:
            dihedral_types[dtype] = [dihedral]

    prior_dihedral = {}

    for dihedral in dihedral_types.keys():
        dihedral_vals = []
        for idx0, idx1, idx2, idx3 in dihedral_types[dihedral]:
            vec1 = minimum_image_distance(mol.coords[idx1, :,:] - mol.coords[idx0,:, :], box_size)
            vec2 = minimum_image_distance(mol.coords[idx2, :,:] - mol.coords[idx1,:, :], box_size)
            vec3 = minimum_image_distance(mol.coords[idx3, :,:] - mol.coords[idx2,:, :], box_size)

            norm1 = np.cross(vec1.T, vec2.T).T
            norm2 = np.cross(vec2.T, vec3.T).T
            
            cosine_dihedral = np.einsum('ij,ij->j', norm1, norm2) / (np.linalg.norm(norm1, axis=0) * np.linalg.norm(norm2, axis=0))
            dihedral_vals.extend(np.degrees(np.arccos(np.clip(cosine_dihedral, -1.0, 1.0))))  # in degrees

        dihedral_range = [np.min(dihedral_vals), np.max(dihedral_vals)]
        counts, bins = np.histogram(dihedral_vals, bins=num_bins, range=dihedral_range)
        mask = counts > 0
        p_phi = counts / counts.sum()
        bins_center = 0.5 * (bins[1:] + bins[:-1])
        p_phi = p_phi[mask]
        bins_center = bins_center[mask]
        dG_nz = -kB * Temp * np.log(p_phi)
        
        def multi_harmonic_dihedral(x, *params):
            n_harmonic = len(params)
            E = 0
            for i in range(n_harmonic):
                E += params[i] * np.cos(np.radians(x))**i
            return E
        
        p0=[0.1] * n_harmonic
        
        popt, _ = curve_fit(multi_harmonic_dihedral, bins_center, dG_nz, p0=p0,
                            maxfev=1000000
                            )

        dname = f"({dihedral[0]}, {dihedral[1]}, {dihedral[2]}, {dihedral[3]})"
        prior_dihedral[dname] = {'A_i': popt[:n_harmonic].tolist()}

    return prior_dihedral

In [ ]:
import json
repulsion = get_param_repulsion(mol,cutoff=16)
bonds = get_param_bond(mol,n_gaussians=1)
angles = get_param_angle(mol,n_gaussians=3)
dihedrals = get_param_dihedral(mol,n_harmonic=8)

with open('../data/CG/prior.json', 'w') as f:
    json.dump({'repulsion': repulsion, 'bonds': bonds, 'angles': angles, 'dihedrals': dihedrals}, f)

print(bonds)
print(angles)
print(dihedrals)